In [2]:
import nearest_neighbors as nn
import cross_validation as cross_val
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

In [3]:
mnist = fetch_openml('mnist_784')

In [4]:
X = mnist.data
y = mnist.target.astype(int)
X.shape, y.shape

((70000, 784), (70000,))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000, shuffle=True, random_state=42)

## Задание №1

In [ ]:
num_features = [10, 20, 100]
select_features = [np.random.choice(np.arange(784), i, replace=False) for i in num_features]

strategy = ["my_own", "brute", "kd_tree", "ball_tree"]
metric = "euclidean"

time = np.zeros((len(strategy), len(num_features)))

ls = ["-", "--", "-.", ":"]
plt.figure(figsize=(10, 6))
for i, strat in enumerate(strategy):
    model = nn.KNNClassifier(5, strategy=strat, metric=metric)
    for j, f in enumerate(select_features):
        model.fit(X_train[:, f], y)
        t = %timeit -q -o model.find_kneighbors(X_test[:, f])
        time[i][j] = t.average
    plt.plot(num_features, time[i], label=strat, ls=ls[i])

plt.ylabel("Время с.", size=12)
plt.xlabel("Размерность пространства", size=12)
plt.yticks(np.linspace(time.min(), time.max(), 10).round(4))
plt.xticks(np.arange(10, 101, 10))
plt.title("Время работы в зависимости от размерности пространства", size=14)
plt.grid(True, color="lightgrey")
plt.legend(fontsize=12)
plt.show()

## Задание №2

In [ ]:
X=X[:10000,:]
y=y[:10000]

In [ ]:
str_w = "True" if w else "False"
rg = np.arange(1, 11)
plt.figure(figsize=(10, 6))
for w in [False, True]:

    acc = cross_val.knn_cross_val_score(X, y, k_list=rg,
                                        cv=cross_val.kfold(X.shape[0], 3),
                                        strategy="brute", weights=w)
    plt.plot(rg, [sum(x[1])/len(x[1])for x in acc.items()],
             ls="-.",label="euclidean, weight="+str_w)

    acc = cross_val.knn_cross_val_score(X, y, k_list=rg,
                                        cv=cross_val.kfold(X.shape[0], 3),
                                        metric="cosine", weights=w)
    plt.plot(rg, [sum(x[1])/len(x[1])for x in acc.items()],
             ls="--", label="cosine, weight="+str_w)

plt.xticks(range(1, 11))
plt.xlabel("количество соседей", size=12)
plt.ylabel("точность", size=12)
plt.title("Точность в зависимости от количества соседей", size=14)
plt.grid(True, color="lightgrey")
plt.legend(fontsize=12)
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,train_size=0.5, shuffle=True)

model_time_euc = nn.KNNClassifier(3, metric="euclidean", test_block_size=500)
model_time_cosine = nn.KNNClassifier(3, metric="cosine", test_block_size=500)
model_time_euc.fit(X_train,y_train)
model_time_cosine.fit(X_train,y_train)

time_euclidean = %timeit -q -o -n 1 model_time_euc.predict(X_test)
time_coisne = %timeit -q -o -n 1 model_time_cosine.predict(X_test)

In [ ]:
time_euclidean.average,time_coisne.average